In [1]:
%load_ext autoreload
%autoreload 2

# Extract Time Indicator

In [24]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

In [25]:
time = gdf[gdf['item_type'] == 'building'][['id','ts']]
time['last_edit(months)'] = [diff_month(datetime.now(), row) for row in time['ts']]

In [26]:
time

,id,ts,last_edit(months)
0,2402565,2019-01-25 15:01:07+00:00,43
1,2514001,2019-05-19 04:37:30+00:00,39
2,2959361,2017-04-09 13:39:57+00:00,64
3,2959362,2017-04-09 13:39:57+00:00,64
4,2959364,2017-04-09 13:39:58+00:00,64
...,...,...,...
140736,1064791239,2022-05-29 07:43:21+00:00,3
140737,1064791240,2022-05-29 07:43:21+00:00,3
140738,1070775598,2022-06-17 22:02:40+00:00,2
140739,1071164414,2022-06-19 03:46:38+00:00,2


In [31]:
stats = time.describe()[['last_edit(months)']]
stats.filter(items = ['mean', '50%'], axis=0)

,last_edit(months)
mean,33.911746
50%,32.000000


In [ ]:
# If last updated was a long time ago, the paper defines it as more reliable. Seems a bit contradictory! (Shouldn't it be more reliable as we have more new updates )

In [2]:
from extract_indirect_indicators import *

In [3]:
indir_ind = extract_indirect_indicators('rec')
indir_ind

,cell_no,bid,road_cnt,road_uc,road_le_time,building_cnt,building_uc,building_le_time,poi_cnt,poi_uc,poi_le_time
0,0,"[649762976, 649762978, 649762980, 649762982, 8...",28,14,12,23,3,7,0,0,0
1,1,"[649762910, 649762914, 649762916, 649762919, 6...",54,22,8,56,3,22,2,2,70
2,2,"[683804184, 751199360, 751199361, 751199362, 7...",90,24,7,690,2,7,3,2,7
3,3,"[11243064, 12497977, 483259351, 483260229, 484...",157,41,5,996,11,5,45,8,5
4,4,"[590299017, 742250779, 742250780, 742250781, 7...",80,23,10,696,1,20,5,2,40
...,...,...,...,...,...,...,...,...,...,...,...
377,377,[885994795],16,19,8,1,1,20,0,0,0
378,378,"[885900392, 885900397]",7,9,13,2,1,20,0,0,0
379,379,"[153236225, 153236231, 153252396, 153252972, 4...",17,21,9,7,2,71,0,0,0
380,380,[681001036],22,25,12,1,1,40,2,2,26


In [ ]:
# Instead of storing the pbf files for history orlatest osm
# extract and store nodes, ways, relations (for latest osm data)
# and also other important attributes required (required for dir/indir indicators) for hisotrical data 
# create dataframes and store them in feather files!!

# And then extract them for direct or indirect indicators and use accordingly (Have to change a lot of code for that!!)

In [ ]:
stats = get_stats(indir_ind)
stats

,road_cnt,road_uc,building_cnt,building_uc,building_le_time,poi_cnt,poi_uc,poi_le_time
mean,100.356113,37.240353,420.692236,8.147373,9.445374,17.160855,7.992562,9.806137
25%,50.000000,27.000000,8.000000,4.000000,2.000000,3.000000,3.000000,0.000000
50%,81.000000,36.000000,81.000000,6.000000,4.000000,9.000000,6.000000,2.000000
